In [3]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', 'LM(영어)', 'LM(중국어)', 'LM(일본어)', '번역(한-영)', '번역(한-중)', '번역(한-일)']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
langs = ['en', 'ch', 'jp']


#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            sentences = line['sentences']
            source_text = [s['src_sentence'] for s in sentences]
            target_text = [s['tgt_sentence'] for s in sentences]
            source_language = sentences[0]['src_lang']
            target_language = sentences[0]['tgt_lang']
            #### data preprocess end 
            
            #### LM(한국어)
            data = {'text': None}
            ## preprocess data from line
            for text in source_data:
                data['text'] = text
                ## preprocess data from line end
                task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### LM(한국어) end
            
            
            for i, lang in enumerate(langs):
                if lang == target_language:
                    for text in target_text:
                        data['text'] = text
                        #### LM
                        task_files[i+1].write(json.dumps(data, ensure_ascii=False)+'\n')
                        # print(json.dumps(data, indent=4, ensure_ascii=False))
                        #### LM end
            
            data = {'input': {}, 'output': None}
            data['input']['source_language'] = source_language
            data['input']['target_language'] = target_language
            
            for i, lang in enumerate(langs):
                if lang == target_language:
                    for source, target in zip(source_text, target_text):
                        data['input']['source_text'] = source
                        data['output'] = target
                        #### 번역(한-영)
                        task_files[i+4].write(json.dumps(data, ensure_ascii=False)+'\n')
                        # print(json.dumps(data, indent=4, ensure_ascii=False))
                        #### 번역(한-영) end
            
    
            
            # break
        # break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 3/3 [00:00<00:00,  8.95it/s]
